# Machine Learning: Text Classification Assignment

In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split as tts
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader

### Use the CategorizedPlaintextCorpusReader to import the AP_News corpus.

In [49]:
DOC_PATTERN = r'.*\.txt'
CAT_PATTERN = r'([\w_\s]+)/.*'
path = '/content/drive/MyDrive/content/AP_News'
corpus = CategorizedPlaintextCorpusReader(path, DOC_PATTERN, cat_pattern = CAT_PATTERN)

### Create two separate lists - one containing the text from each document and another containing the category of each article in the corpus.

In [50]:
docs = [corpus.raw(fileid) for fileid in corpus.fileids()]

categories = [corpus.categories(fileid)[0] for fileid in corpus.fileids()]

In [51]:
categories

['health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'health',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 'politics',
 

In [52]:
docs

['HONOLULU (AP) — The University of Hawaii seeks additional funds for student mental health services, scholarships and other items in a new supplementary budget request, officials said. The Board of Regents approved the fiscal year 2020-2021 supplemental operating budget of about $28 million Thursday, The Honolulu Star-Advertiser reported. The request will be submitted to the state Legislature and Democratic Gov. The university requested $2.6 million to hire 19 psychologists for the 10-campus system. The University of Hawaii Manoa has eight psychologists, the Hilo campus has three, and the West Oahu campus has 1.75 positions, while each community college has one position, said Allyson Tanouye, who coordinates mental health throughout the university system. "The national standard is one mental health professional per 1,000 to 1,500 students," Tanouye said. "If we add the 19 positions we will be up to one per 1,500. That\'s how low we are". The mental health funding would also expand pro

### Preprocess the corpus, ensuring to include the following steps.

- Word tokenize the documents.
- Lemmatize, stem, and lowercase all tokens.
- Remove punctuation and stop words.

In [62]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
def preprocess(docs):
  lemmatizer = WordNetLemmatizer()
  stemmer = SnowballStemmer('english')
  preprocessed = []

  for doc in docs:
    tokenized = word_tokenize(doc)

    cleaned = [stemmer.stem(lemmatizer.lemmatize(token.lower()))
               for token in tokenized
               if not token.lower() in stopwords.words('english')
               if token.isalpha()]

    untokenized = " ".join(cleaned)
    preprocessed.append(untokenized)

  return preprocessed

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [63]:
docs = preprocess(docs)

### Split the data into training and testing sets with the size of the test set being 30% of the records.

In [64]:
X_train, X_test, y_train, y_test = tts(docs, categories, test_size=0.3)

### Construct a pipeline that TF-IDF vectorizes the text and trains a Random Forest classification model.

In [68]:
from sklearn.linear_model import LogisticRegression
model = Pipeline([
       ('vect', CountVectorizer()),
       ('tfidf', TfidfTransformer()),
       ('rfc', RandomForestClassifier()),           
])

model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

### Generate predictions on the test set and print a classification report to evaluate how well the model performed.

In [69]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

      health       0.61      0.73      0.67        15
    politics       0.81      0.76      0.79        17
      sports       0.93      0.93      0.93        15
        tech       0.76      0.68      0.72        19

    accuracy                           0.77        66
   macro avg       0.78      0.78      0.78        66
weighted avg       0.78      0.77      0.77        66



### Perform 10-fold cross validation and obtain the averge F1 score across all the folds.

In [72]:
scores = cross_val_score(model, docs, categories, cv=10, scoring='f1_macro')
scores.mean()

0.7854180395627764

### Ingest, preprocess, and predict the topic of the article at the following URL.

In [74]:
url = 'https://www.nytimes.com/2019/11/25/business/uber-london.html'

In [78]:
import requests
from bs4 import BeautifulSoup
def get_url_text(url):
  response = requests.get(url)
  content = response.text

  TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li']
  soup = BeautifulSoup(content, 'lxml')
  text_list = [tag.get_text() for tag in soup.find_all(TAGS)]
  text = ' '.join(text_list)
  return text

text = get_url_text(url)
cleaned = preprocess([text])
model.predict(cleaned)[0]

'tech'